In [3]:
from fastai.vision.all import *

/Users/bruceantley/anaconda3/envs/fastai310/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [4]:
!which python
!python --version

/Users/bruceantley/anaconda3/envs/fastai310/bin/python
Python 3.10.18


In [5]:
from pathlib import Path

base_path = Path('/Users/bruceantley/Documents/face_classifier_310/fine_tune_sets/round1')
model_path = Path('/Users/bruceantley/Documents/face_classifier_310/model_exports/round0_final.pkl')

In [6]:
dls = ImageDataLoaders.from_folder(
    base_path,
    train='.',                  # real and fake are both directly inside round1/
    valid_pct=0.2,              # 20% of each class held out for validation
    item_tfms=Resize(256),      # Resize all to 256x256
    batch_tfms=[                # Data augmentation + normalization
        *aug_transforms(),
        Normalize.from_stats(*imagenet_stats)
    ],
    bs=16,
    seed=42
)

In [7]:
learn = cnn_learner(dls, resnet50)
learn = load_learner(model_path)
learn.dls = dls  # override the data loaders with your fine-tune set

/Users/bruceantley/anaconda3/envs/fastai310/lib/python3.10/site-packages/fastai/vision/learner.py:303: UserWarning: `cnn_learner` has been renamed to `vision_learner` -- please update your code
  warn("`cnn_learner` has been renamed to `vision_learner` -- please update your code")
/Users/bruceantley/anaconda3/envs/fastai310/lib/python3.10/site-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")


In [8]:
learn.fine_tune(6)

epoch,train_loss,valid_loss,accuracy,time
0,1.562586,1.558640,0.589744,00:15


epoch,train_loss,valid_loss,accuracy,time
0,1.017276,1.265402,0.743590,00:14
1,0.814438,1.011015,0.769231,00:14
2,0.680540,0.809660,0.846154,00:14
3,0.569762,0.741226,0.846154,00:14
4,0.492821,0.723452,0.846154,00:14
5,0.430080,0.682136,0.846154,00:14


In [9]:
learn.unfreeze()
learn.fit_one_cycle(5, lr_max=slice(1e-6, 1e-4))

epoch,train_loss,valid_loss,accuracy,time
0,0.266487,0.676702,0.820513,00:15
1,0.266305,0.699240,0.846154,00:14
2,0.241891,0.714704,0.846154,00:14
3,0.225493,0.707648,0.846154,00:14
4,0.221040,0.772144,0.846154,00:14


In [10]:
learn.fit_one_cycle(2, lr_max=slice(1e-6, 1e-4), wd=0.01)

epoch,train_loss,valid_loss,accuracy,time
0,0.164740,0.780101,0.846154,00:14
1,0.192978,0.718835,0.846154,00:14


In [11]:
# 🧪 Predict on a new image
#fake gym
img = PILImage.create('/Users/bruceantley/Desktop/Screenshot 2025-07-06 at 3.38.45 PM.png')  # replace with an actual file
pred, pred_idx, probs = learn.predict(img)
print(f"Prediction: {pred}; Probability: {probs[pred_idx]:.04f}")

Prediction: fake; Probability: 0.9073


In [12]:
# 🧪 Predict on a new image
#ariel
img = PILImage.create('/Users/bruceantley/Desktop/Screenshot 2025-07-06 at 3.35.04 PM.png')  # replace with an actual file
pred, pred_idx, probs = learn.predict(img)
print(f"Prediction: {pred}; Probability: {probs[pred_idx]:.04f}")

Prediction: real; Probability: 0.7741


In [13]:
learn.export("/Users/bruceantley/Documents/face_classifier_310/model_exports/round1_final.pkl")